# Project Data Science: Bitcoin Price Prediction

## 1. Introduction
In this day AI have a significant impact almost in our life and work place, and for this field we will leveraging ai in sequential data development. In this Data Science Project we will do some research about BTCUSD consolidation price with Deep Learning Neural Network LSTM method using PyTorch library
### 1. 1 Project Objective
The main objective is to predict BTC price until the next months with LSTM (Long Short Term Memory) method
### 1. 2 Dataset Description
`btcusd_1-min_data` is the name of the dataset, which was taken from the Kaggle Dataset source: https://www.kaggle.com/datasets/mczielinski/bitcoin-historical-data and has seven columns: `Timestamps, Open, Close, Min, Max, Datetime`. The data is a compilation of the price of Bitcoin from 2012 until early 2025, which is a changes every minute. 

## 2. Data Preparation


### 2.1 Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('ggplot')

### 2.2 Loading the Dataset

In [6]:
df = pd.read_csv('btcusd_1-min_data.csv', parse_dates=['datetime'])

### 2.3 Initial Data Exploration

In [11]:
# get 5 first data
df.head()

,Timestamp,Open,High,Low,Close,Volume,datetime
0,1.325412e+09,4.58,4.58,4.58,4.58,0.0,2012-01-01 10:01:00+00:00
1,1.325412e+09,4.58,4.58,4.58,4.58,0.0,2012-01-01 10:02:00+00:00
2,1.325412e+09,4.58,4.58,4.58,4.58,0.0,2012-01-01 10:03:00+00:00
3,1.325412e+09,4.58,4.58,4.58,4.58,0.0,2012-01-01 10:04:00+00:00
4,1.325412e+09,4.58,4.58,4.58,4.58,0.0,2012-01-01 10:05:00+00:00


In [ ]:
# get data information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7008205 entries, 0 to 7008204
Data columns (total 7 columns):
 #   Column     Dtype              
---  ------     -----              
 0   Timestamp  float64            
 1   Open       float64            
 2   High       float64            
 3   Low        float64            
 4   Close      float64            
 5   Volume     float64            
 6   datetime   datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), float64(6)
memory usage: 374.3 MB


In [12]:
# to know mean, max, min, std, 25%, 50%, 75% of each column
df.describe()

,Timestamp,Open,High,Low,Close,Volume
count,7.008205e+06,7.008205e+06,7.008205e+06,7.008205e+06,7.008205e+06,7.008205e+06
mean,1.535659e+09,1.737403e+04,1.738098e+04,1.736687e+04,1.737403e+04,5.303707e+00
std,1.213868e+08,2.401468e+04,2.402270e+04,2.400646e+04,2.401466e+04,2.252397e+01
min,1.325412e+09,3.800000e+00,3.800000e+00,3.800000e+00,3.800000e+00,0.000000e+00
25%,1.430535e+09,4.242000e+02,4.243500e+02,4.240300e+02,4.242000e+02,1.814380e-02
50%,1.535658e+09,6.587830e+03,6.590610e+03,6.584380e+03,6.587800e+03,4.692601e-01
75%,1.640781e+09,2.726900e+04,2.727600e+04,2.726300e+04,2.726900e+04,3.035527e+00
max,1.745974e+09,1.091110e+05,1.093560e+05,1.087940e+05,1.090360e+05,5.853852e+03


## 3. Data Cleaning

### 3.1 Handling Missing Values

In [13]:
#find missing valuee
df.isnull().sum()

Timestamp         0
Open              0
High              0
Low               0
Close             0
Volume            0
datetime     225925
dtype: int64

In [14]:
# drop all missing values
df.dropna(inplace=True)
# check again
df.isnull().sum()

Timestamp    0
Open         0
High         0
Low          0
Close        0
Volume       0
datetime     0
dtype: int64

### 3.2 Removing Duplicate Values

In [19]:
# check duplicate values without Timestamp column
df.iloc[:,1:].duplicated().sum()


np.int64(0)

### 3.3 Change To Daily Dataframe

In [23]:
# change data to daily data 
df_daily = df.resample('D', on='datetime').agg({
    'Volume': 'sum', 
    'High': 'max', 
    'Low': 'min', 
    'Open': 'first', 
    'Close': 'last'
    }).reset_index()
df_daily.head()


,datetime,Volume,High,Low,Open,Close
0,2012-01-01 00:00:00+00:00,10.000000,4.84,4.58,4.58,4.84
1,2012-01-02 00:00:00+00:00,10.100000,5.00,4.84,4.84,5.00
2,2012-01-03 00:00:00+00:00,107.085281,5.32,5.00,5.00,5.29
3,2012-01-04 00:00:00+00:00,107.233260,5.57,4.93,5.29,5.57
4,2012-01-05 00:00:00+00:00,70.328742,6.46,5.57,5.57,6.42


In [ ]:
# change datatime data to date and convert to datetime
df_daily['date'] = df_daily['datetime'].dt.date
df_daily['date'] = pd.to_datetime(df_daily['date'])
# drop datetime column
df_daily.drop(columns=['datetime'], inplace=True, axis=1)
# chech data types
df_daily.dtypes
# check the first 5 data
df_daily.head()

,Volume,High,Low,Open,Close,date
0,10.000000,4.84,4.58,4.58,4.84,2012-01-01
1,10.100000,5.00,4.84,4.84,5.00,2012-01-02
2,107.085281,5.32,5.00,5.00,5.29,2012-01-03
3,107.233260,5.57,4.93,5.29,5.57,2012-01-04
4,70.328742,6.46,5.57,5.57,6.42,2012-01-05


In [31]:
# check min and max of each column
df_daily.min(), df_daily.max()

(Volume                    0.0
 High                     4.38
 Low                       3.8
 Open                     4.38
 Close                    4.38
 date      2012-01-01 00:00:00
 dtype: object,
 Volume          127286.486533
 High                 109030.0
 Low                  106187.0
 Open                 108314.0
 Close                106187.0
 date      2025-03-15 00:00:00
 dtype: object)

In [33]:
# chech total data
df_daily.shape[0]

4823

In [34]:
# export data to csv file
df_daily.to_csv('btcusdt_daily.csv', index=False)

## 4. Exploratory Data Analysis (EDA)